# Spectrum Translation Benchmark

This notebooks calculates the metrics of concordance on paired OuluCasia dataset of 2 models - VIS model and NIR model.

***

In [6]:
import pandas as pd
import os
import random
import math
import numpy as np
from sklearn.metrics import cohen_kappa_score


from skeleton.inference import Inference

Define functions

In [7]:
def compute_metrics(pred_class_vl,
                   pred_class_ni,
                   pred_val_vl,
                   pred_val_ni,
                   pred_aro_vl,
                   pred_aro_ni,
                   expr_classes=None):
    def compute_ccc(x,y):
        ''' Concordance Correlation Coefficient'''
        sxy = np.sum((x - x.mean())*(y - y.mean()))/x.shape[0]
        rhoc = 2*sxy / (np.var(x) + np.var(y) + (x.mean() - y.mean())**2)
        return rhoc

    concordance = np.mean(np.array(pred_class_vl) == np.array(pred_class_ni))
    print(f"Concordance: {concordance * 100}%")

    kappa = cohen_kappa_score(pred_class_vl, pred_class_ni)
    print(f"Cohen's Kappa: {kappa}")

    ccc = compute_ccc(pred_val_vl, pred_val_ni)
    print(f"valence CCC: {ccc}")

    ccc = compute_ccc(pred_aro_vl, pred_aro_ni)
    print(f"arousal CCC: {ccc}")

    avg_distance_val = np.mean(np.abs(pred_val_vl - pred_val_ni))
    print(f'Average distance valence: {avg_distance_val}')

    avg_distance_aro = np.mean(np.abs(pred_aro_vl - pred_aro_ni))
    print(f'Average distance arousal: {avg_distance_aro}')
    
    std_dev_val = np.std(pred_val_vl - pred_val_ni)
    print(f'Standard deviation valence: {std_dev_val}')
    
    std_dev_aro = np.std(pred_aro_vl - pred_aro_ni)
    print(f'Standard deviation arousal: {std_dev_aro}')
    
    if expr_classes is not None:
        print("="*30 + "\n" + "="*30)
        classes = np.unique(expr_classes)
        for c in classes:
            print('\n' + "*"*20 + " CLASS: " + str(c) + '\n')
            compute_metrics(
                pred_class_vl[expr_classes==c],
                pred_class_ni[expr_classes==c],
                pred_val_vl[expr_classes==c],
                pred_val_ni[expr_classes==c],
                pred_aro_vl[expr_classes==c],
                pred_aro_ni[expr_classes==c],
            )
        
    
def calculate_outputs(df_joined, inf_vl, inf_ni):
    filepath_vl = []
    filepath_ni = []
    out_vl = []
    out_ni = []
    for i, (vl_fp, ni_fp) in enumerate(zip(df_joined.filepath, df_joined.filepath_ni)):
        if vl_fp.split("/")[-1] == "Thumbs.db":
            continue
        print(i, ni_fp, vl_fp)
        filepath_vl.append(vl_fp)
        filepath_ni.append(ni_fp)
        out_vl.append(inf_vl.infer_instant_from_filenames([vl_fp]))
        out_ni.append(inf_ni.infer_instant_from_filenames([ni_fp]))
    
    return filepath_vl, filepath_ni, out_vl, out_ni

fetch data

In [8]:
df_oulucasia = pd.read_csv("annotations_splits/df_oulucasia_all.csv")
df_oulucasia_vl = df_oulucasia[df_oulucasia["spectra"]=="vis"]
df_oulucasia_ni = df_oulucasia[df_oulucasia["spectra"]=="nir"]
df_oulucasia_ni.head()

df_oulucasia_ni.columns = [str(col) + '_ni' for col in df_oulucasia_ni.columns]

# Perform inner join on 'adjusted_unique_filename' column
df_joined = pd.merge(df_oulucasia_vl, df_oulucasia_ni, left_on='adjusted_unique_filename', right_on='adjusted_unique_filename_ni', how='inner')

define inference objects for NIR and VIS

In [11]:
settings = {
    "face_detector": {
        "net_type": Inference.net_type.FACE_DETECTOR_CENTERFACE,
        "retina_face_align": True,
        "remove_black_stripes": True,
        "display_images": False,
        "save_image_to_folder": None,
    },
    "spectrum_translator": {
        "net_type": None,
        "pth_to_onnx": 'models/spectrum_translation/split_w_all_w_only_aff_and_buua_in_vis_lambda_A_15-GA-21epoch.onnx',
        "input_as_avg_grayscale": False,
        "output_as_avg_grayscale": False,
        "display_images": True,
        "save_image_to_folder": "transtmp",
    },
    "fer": {
        "net_type": Inference.net_type.FER_MOBILENET,
        "pth_to_onnx": 'models/mobilenet_on_aff.onnx',
        "display_images": False,
        "save_image_to_folder": None,
    }
}

inf_vl = Inference(settings, None)

settings["spectrum_translator"]["net_type"] = Inference.net_type.SPECTRUM_TRANSLATOR_ORIG_CYCLEGAN
inf_ni = Inference(settings, None)

Using '{'net_type': <net_type.FACE_DETECTOR_CENTERFACE: 'F'>, 'retina_face_align': True, 'remove_black_stripes': True, 'display_images': False, 'save_image_to_folder': None}' as face detector model
Using '{'net_type': <net_type.FER_MOBILENET: 'E'>, 'pth_to_onnx': 'models/mobilenet_on_aff.onnx', 'display_images': False, 'save_image_to_folder': None}' as FER model
Using '{'net_type': <net_type.FACE_DETECTOR_CENTERFACE: 'F'>, 'retina_face_align': True, 'remove_black_stripes': True, 'display_images': False, 'save_image_to_folder': None}' as face detector model
Using '{'net_type': <net_type.SPECTRUM_TRANSLATOR_ORIG_CYCLEGAN: 'B'>, 'pth_to_onnx': 'models/spectrum_translation/split_w_all_w_only_aff_and_buua_in_vis_lambda_A_15-GA-21epoch.onnx', 'input_as_avg_grayscale': False, 'output_as_avg_grayscale': False, 'display_images': True, 'save_image_to_folder': 'transtmp'}' as spectrum transfer model
Using '{'net_type': <net_type.FER_MOBILENET: 'E'>, 'pth_to_onnx': 'models/mobilenet_on_aff.onnx', 

In [ ]:
filepath_vl, filepath_ni, out_vl, out_ni = calculate_outputs(df_joined, inf_vl, inf_ni)

## Calculate on All data of OuluCasia

Calculates on all OuluCasia data

In [ ]:
df_joined.head(1)

In [ ]:
pred_class_vl = np.array([np.argmax(out[0][2][0][0][0][0]) for out in out_vl])
pred_class_ni = np.array([np.argmax(out[0][2][0][0][0][0]) for out in out_ni])
pred_val_vl = np.array([out[0][2][0][0][1][0][0] for out in out_vl])
pred_val_ni = np.array([out[0][2][0][0][1][0][0] for out in out_ni])
pred_aro_vl = np.array([out[0][2][0][0][1][0][1] for out in out_vl])
pred_aro_ni = np.array([out[0][2][0][0][1][0][1] for out in out_ni])

# expr_classes = df_joined.expression.to_numpy()
expr_classes = np.array([e[1][-1] for e in df_joined.iterrows() if e[1][3].split('.')[-1]!= 'db'])

print(pred_class_vl.shape)

In [ ]:
compute_metrics(pred_class_vl, pred_class_ni, pred_val_vl, pred_val_ni, pred_aro_vl, pred_aro_ni, expr_classes=expr_classes)
print("-"*20)

## Calculate on Top-half data of OuluCasia

***

Calculation of metrics is conducted on the top-half of oulucasia. Under the top-half is meant the upper half of images in emotion of patient set. Effectively, those images are more affected images since each set contains sequence of images that start from neutral to the most affected emotion.

Note that the sequences are note necessarily linear.

In [ ]:
df = df_joined.copy()

# Assuming df is your DataFrame
df = df.sort_values(['oulucasia-patient', 'oulucasia-emotion', 'filename']) # Ensure the data is sorted

# Create a copy of the original DataFrame's index
original_index = df.index

# Group by 'patient' and 'expression', then drop the first half of each group
df = df.groupby(['oulucasia-patient', 'oulucasia-emotion']).apply(lambda x: x[int(np.ceil(x.shape[0]/2.)):]).reset_index(drop=True)

# Get the indices of the remaining samples in the original DataFrame
# remaining_indices = df.index.intersection(original_index)
indices = df_joined[df_joined['filepath'].isin(df.filepath)].index.to_numpy()
indices = indices[:-1]

In [ ]:
pred_class_vl = pred_class_vl[indices]
pred_class_ni = pred_class_ni[indices]
pred_val_vl = pred_val_vl[indices]
pred_val_ni = pred_val_ni[indices]
pred_aro_vl = pred_aro_vl[indices]
pred_aro_ni = pred_aro_ni[indices]

expr_classes = np.array([e[1][-1] for e in df.iterrows() if e[1][3].split('.')[-1]!= 'db'])

print(pred_class_vl.shape)

In [ ]:
compute_metrics(pred_class_vl, pred_class_ni, pred_val_vl, pred_val_ni, pred_aro_vl, pred_aro_ni, expr_classes=expr_classes)

## Calculate on Top-quarter data of OuluCasia

***

Calculation of metrics is conducted on the top-quarter of oulucasia. Under the top-quarter is meant the upper quarter of images in emotion of patient set. Essentially, those images from top-quarter are more affected images since each set contains sequence of images that start from neutral to the most affected emotion.

Note that the sequences are note necessarily linear.

In [ ]:
df = df_joined.copy()

# Assuming df is your DataFrame
df = df.sort_values(['oulucasia-patient', 'oulucasia-emotion', 'filename']) # Ensure the data is sorted

# Create a copy of the original DataFrame's index
original_index = df.index

# Group by 'patient' and 'expression', then drop the first half of each group
df = df.groupby(['oulucasia-patient', 'oulucasia-emotion']).apply(lambda x: x[int(np.ceil(3*x.shape[0]/4.)):]).reset_index(drop=True)

# Get the indices of the remaining samples in the original DataFrame
# remaining_indices = df.index.intersection(original_index)
indices = df_joined[df_joined['filepath'].isin(df.filepath)].index.to_numpy()
indices = indices[:-1]

In [ ]:
pred_class_vl = np.array([np.argmax(out[0][2][0][0][0][0]) for out in out_vl])
pred_class_ni = np.array([np.argmax(out[0][2][0][0][0][0]) for out in out_ni])
pred_val_vl = np.array([out[0][2][0][0][1][0][0] for out in out_vl])
pred_val_ni = np.array([out[0][2][0][0][1][0][0] for out in out_ni])
pred_aro_vl = np.array([out[0][2][0][0][1][0][1] for out in out_vl])
pred_aro_ni = np.array([out[0][2][0][0][1][0][1] for out in out_ni])

pred_class_vl = pred_class_vl[indices]
pred_class_ni = pred_class_ni[indices]
pred_val_vl = pred_val_vl[indices]
pred_val_ni = pred_val_ni[indices]
pred_aro_vl = pred_aro_vl[indices]
pred_aro_ni = pred_aro_ni[indices]

expr_classes = np.array([e[1][-1] for e in df.iterrows() if e[1][3].split('.')[-1]!= 'db'])

print(pred_class_vl.shape)

In [ ]:
compute_metrics(pred_class_vl, pred_class_ni, pred_val_vl, pred_val_ni, pred_aro_vl, pred_aro_ni, expr_classes=expr_classes)

## Results:



### Experiment1.1.0 - retinaface

#### All:
* Concordance: 57.8507277794983%
* Cohen's Kappa: 0.5025220215833031
* valence CCC: 0.8663356579226443
* arousal CCC: 0.734362497816338
* Average distance valence: 0.05879761651158333
* Average distance arousal: 0.06024966388940811
* Standard deviation valence: 0.07865650206804276
* Standard deviation arousal: 0.07647916674613953

******************** CLASS: Anger

9687 9687 9687
* Concordance: 55.14579759862779%
* Cohen's Kappa: 0.402516450397984
* valence CCC: 0.6347016983908523
* arousal CCC: 0.6293452896446139
* Average distance valence: 0.05220651626586914
* Average distance arousal: 0.06477273255586624
* Standard deviation valence: 0.06643147021532059
* Standard deviation arousal: 0.0722218006849289

******************** CLASS: Disgust

9687 9687 9687
* Concordance: 61.517367458866545%
* Cohen's Kappa: 0.38798463364042135
* valence CCC: 0.6871445538236037
* arousal CCC: 0.5713738287339922
* Average distance valence: 0.05363545939326286
* Average distance arousal: 0.05318139120936394
* Standard deviation valence: 0.07295463979244232
* Standard deviation arousal: 0.06590928137302399

******************** CLASS: Fear

9687 9687 9687
* Concordance: 42.92845257903494%
* Cohen's Kappa: 0.28709341070727645
* valence CCC: 0.7375854131238321
* arousal CCC: 0.7034233150182165
* Average distance valence: 0.07149316370487213
* Average distance arousal: 0.06309092789888382
* Standard deviation valence: 0.08587801456451416
* Standard deviation arousal: 0.08322911709547043

******************** CLASS: Happy

9687 9687 9687
* Concordance: 68.44331641285956%
* Cohen's Kappa: 0.3783491112746715
* valence CCC: 0.735362167570651
* arousal CCC: 0.4731181505584242
* Average distance valence: 0.08416485786437988
* Average distance arousal: 0.0485316701233387
* Standard deviation valence: 0.11142238229513168
* Standard deviation arousal: 0.060202695429325104

******************** CLASS: Neutral

9687 9687 9687
* Concordance: 57.97205302758868%
* Cohen's Kappa: 0.35841482058368235
* valence CCC: 0.670006258564099
* arousal CCC: 0.5528289660826563
* Average distance valence: 0.05075392499566078
* Average distance arousal: 0.06025362387299538
* Standard deviation valence: 0.06613016128540039
* Standard deviation arousal: 0.07231764495372772

******************** CLASS: Sad

9687 9687 9687
* Concordance: 44.899817850637525%
* Cohen's Kappa: 0.29813791908442766
* valence CCC: 0.5886086679552094
* arousal CCC: 0.48687942498433445
* Average distance valence: 0.05387421324849129
* Average distance arousal: 0.07290530949831009
* Standard deviation valence: 0.06972689926624298
* Standard deviation arousal: 0.08935479074716568

******************** CLASS: Surprise

* 9687 9687 9687
* Concordance: 73.83015597920277%
* Cohen's Kappa: 0.30241058149799127
* valence CCC: 0.5934948163716197
* arousal CCC: 0.6167494558507134
* Average distance valence: 0.05528314784169197
* Average distance arousal: 0.05937211215496063
* Standard deviation valence: 0.07182855904102325
* Standard deviation arousal: 0.08473566919565201

#### Top-half
* Concordance: 62.02585293494385%
* Cohen's Kappa: 0.5573767608730061
* valence CCC: 0.9024176092758232
* arousal CCC: 0.7816047313371407
* Average distance valence: 0.059443287551403046
* Average distance arousal: 0.05662716180086136
* Standard deviation valence: 0.0807383581995964
* Standard deviation arousal: 0.07365524023771286

******************** CLASS: Anger

* Concordance: 57.391304347826086%
* Cohen's Kappa: 0.4198996592279475
* valence CCC: 0.6684558996555656
* arousal CCC: 0.6444984736699506
* Average distance valence: 0.0490817092359066
* Average distance arousal: 0.06321152299642563
* Standard deviation valence: 0.06205809488892555
* Standard deviation arousal: 0.0702233612537384

******************** CLASS: Disgust

* Concordance: 67.68617021276596%
* Cohen's Kappa: 0.4485172957018778
* valence CCC: 0.7394528354699241
* arousal CCC: 0.6381400692955534
* Average distance valence: 0.04859618470072746
* Average distance arousal: 0.04744467884302139
* Standard deviation valence: 0.06560366600751877
* Standard deviation arousal: 0.05866306275129318

******************** CLASS: Fear

* Concordance: 44.349939246658565%
* Cohen's Kappa: 0.30028958604046774
* valence CCC: 0.7422471203864279
* arousal CCC: 0.7258900831633898
* Average distance valence: 0.0730057880282402
* Average distance arousal: 0.06081138178706169
* Standard deviation valence: 0.08765219151973724
* Standard deviation arousal: 0.08166056126356125

******************** CLASS: Happy

* Concordance: 75.03105590062111%
* Cohen's Kappa: 0.39193455042878933
* valence CCC: 0.7354416326220617
* arousal CCC: 0.49775275980880945
* Average distance valence: 0.08142605423927307
* Average distance arousal: 0.04823221266269684
* Standard deviation valence: 0.11066902428865433
* Standard deviation arousal: 0.060067035257816315

******************** CLASS: Sad

* Concordance: 44.90616621983914%
* Cohen's Kappa: 0.3062741815055321
* valence CCC: 0.6085364906710639
* arousal CCC: 0.48688081905611547
* Average distance valence: 0.05249401926994324
* Average distance arousal: 0.07394270598888397
* Standard deviation valence: 0.06839045137166977
* Standard deviation arousal: 0.08879763633012772

******************** CLASS: Surprise

* Concordance: 82.74111675126903%
* Cohen's Kappa: 0.38891043039937967
* valence CCC: 0.630538251263814
* arousal CCC: 0.7054698762138026
* Average distance valence: 0.05033693462610245
* Average distance arousal: 0.04647707939147949
* Standard deviation valence: 0.06617076694965363
* Standard deviation arousal: 0.06757413595914841

#### Top-quarter
* Concordance: 65.4480606330807%
* Cohen's Kappa: 0.5973222068640253
* valence CCC: 0.9159484494346315
* arousal CCC: 0.8116541693713143
* Average distance valence: 0.056264135986566544
* Average distance arousal: 0.0537295900285244
* Standard deviation valence: 0.07554922997951508
* Standard deviation arousal: 0.06937969475984573

******************** CLASS: Anger

* Concordance: 58.2901554404145%
* Cohen's Kappa: 0.4300362268996194
* valence CCC: 0.6858917333128943
* arousal CCC: 0.6557496231428979
* Average distance valence: 0.048291489481925964
* Average distance arousal: 0.06024923920631409
* Standard deviation valence: 0.0606718547642231
* Standard deviation arousal: 0.06866452097892761

******************** CLASS: Disgust

* Concordance: 70.2247191011236%
* Cohen's Kappa: 0.5010907360154422
* valence CCC: 0.7868061327579536
* arousal CCC: 0.7241876391190927
* Average distance valence: 0.04709802567958832
* Average distance arousal: 0.045097604393959045
* Standard deviation valence: 0.06263026595115662
* Standard deviation arousal: 0.05367794260382652

******************** CLASS: Fear

* Concordance: 50.63938618925832%
* Cohen's Kappa: 0.37690014780074144
* valence CCC: 0.760731968002046
* arousal CCC: 0.7506965429383967
* Average distance valence: 0.06988585740327835
* Average distance arousal: 0.05860134959220886
* Standard deviation valence: 0.08256994932889938
* Standard deviation arousal: 0.0787895992398262

******************** CLASS: Happy

* Concordance: 80.89005235602095%
* Cohen's Kappa: 0.5280997749310409
* valence CCC: 0.8074436669669379
* arousal CCC: 0.5268222756933223
* Average distance valence: 0.07320651412010193
* Average distance arousal: 0.0468067042529583
* Standard deviation valence: 0.09816835820674896
* Standard deviation arousal: 0.05829058215022087

******************** CLASS: Sad

* Concordance: 46.1756373937677%
* Cohen's Kappa: 0.31921070261272044
* valence CCC: 0.6182029181421687
* arousal CCC: 0.5129415770417955
* Average distance valence: 0.05240326374769211
* Average distance arousal: 0.07272327691316605
* Standard deviation valence: 0.06914619356393814
* Standard deviation arousal: 0.08485103398561478

******************** CLASS: Surprise

* Concordance: 86.13333333333333%
* Cohen's Kappa: 0.5786880995592429
* valence CCC: 0.7519836843684129
* arousal CCC: 0.8548506870536218
* Average distance valence: 0.04534515365958214
* Average distance arousal: 0.0393064059317112
* Standard deviation valence: 0.05776553973555565
* Standard deviation arousal: 0.05461449176073074

### Experiment1.1.0 (centerface)

#### all
Concordance: 59.41984102405286%
Cohen's Kappa: 0.5227833688895651
valence CCC: 0.8691433697956565
arousal CCC: 0.7291357114022443
Average distance valence: 0.0596376433968544
Average distance arousal: 0.06062867119908333
Standard deviation valence: 0.07827653735876083
Standard deviation arousal: 0.07651819288730621

******************** CLASS: Anger

Concordance: 52.5728987993139%
Cohen's Kappa: 0.373055498567299
valence CCC: 0.6245955630730727
arousal CCC: 0.5461274536839178
Average distance valence: 0.05093434825539589
Average distance arousal: 0.07123688608407974
Standard deviation valence: 0.06607680767774582
Standard deviation arousal: 0.07583993673324585

******************** CLASS: Disgust

Concordance: 61.517367458866545%
Cohen's Kappa: 0.3829394426580921
valence CCC: 0.6845204298226277
arousal CCC: 0.5497533868922818
Average distance valence: 0.05494733154773712
Average distance arousal: 0.05604345351457596
Standard deviation valence: 0.0733896940946579
Standard deviation arousal: 0.06917703151702881

******************** CLASS: Fear

Concordance: 51.41430948419301%
Cohen's Kappa: 0.38789244902141173
valence CCC: 0.7699600342117157
arousal CCC: 0.6867120487279221
Average distance valence: 0.06766387075185776
Average distance arousal: 0.06191536784172058
Standard deviation valence: 0.08380664139986038
Standard deviation arousal: 0.08072902262210846

******************** CLASS: Happy

Concordance: 68.61252115059222%
Cohen's Kappa: 0.3730464980291629
valence CCC: 0.7330592193375267
arousal CCC: 0.36743318563173716
Average distance valence: 0.0822698101401329
Average distance arousal: 0.051379065960645676
Standard deviation valence: 0.11130417138338089
Standard deviation arousal: 0.06485734134912491

******************** CLASS: Neutral

Concordance: 58.11537083482623%
Cohen's Kappa: 0.39097672081403234
valence CCC: 0.638291402375926
arousal CCC: 0.5573043244639925
Average distance valence: 0.05453163757920265
Average distance arousal: 0.05933648347854614
Standard deviation valence: 0.06858813762664795
Standard deviation arousal: 0.07318460196256638

******************** CLASS: Sad

Concordance: 49.544626593806925%
Cohen's Kappa: 0.3565982075899774
valence CCC: 0.6199399023910385
arousal CCC: 0.5487112581658459
Average distance valence: 0.052430152893066406
Average distance arousal: 0.06738430261611938
Standard deviation valence: 0.0660829022526741
Standard deviation arousal: 0.08129630237817764

******************** CLASS: Surprise

Concordance: 75.82322357019065%
Cohen's Kappa: 0.3449371724339987
valence CCC: 0.5616469018044182
arousal CCC: 0.6206563980307307
Average distance valence: 0.060543306171894073
Average distance arousal: 0.05908821150660515
Standard deviation valence: 0.07541907578706741
Standard deviation arousal: 0.08436671644449234

#### top-half
Concordance: 63.27611782157236%
Cohen's Kappa: 0.5706212105596171
valence CCC: 0.9091924795545869
arousal CCC: 0.7712209292033916
Average distance valence: 0.05877089127898216
Average distance arousal: 0.05858829244971275
Standard deviation valence: 0.0785200372338295
Standard deviation arousal: 0.0745709240436554


******************** CLASS: Anger

Concordance: 53.54037267080746%
Cohen's Kappa: 0.37290147885857117
valence CCC: 0.6825157483922949
arousal CCC: 0.5522838934456982
Average distance valence: 0.04685927554965019
Average distance arousal: 0.07036659866571426
Standard deviation valence: 0.05993224307894707
Standard deviation arousal: 0.07543367892503738

******************** CLASS: Disgust

Concordance: 67.02127659574468%
Cohen's Kappa: 0.42204041155324157
valence CCC: 0.7409798724490202
arousal CCC: 0.6162096266794793
Average distance valence: 0.04828973114490509
Average distance arousal: 0.05172652751207352
Standard deviation valence: 0.06578581780195236
Standard deviation arousal: 0.06204283609986305

******************** CLASS: Fear

Concordance: 52.61239368165249%
Cohen's Kappa: 0.40097867956642486
valence CCC: 0.7763069679409484
arousal CCC: 0.687712986880044
Average distance valence: 0.06928279250860214
Average distance arousal: 0.06261512637138367
Standard deviation valence: 0.08549821376800537
Standard deviation arousal: 0.08202695846557617

******************** CLASS: Happy

Concordance: 74.78260869565217%
Cohen's Kappa: 0.3978658255739831
valence CCC: 0.7436813723342384
arousal CCC: 0.4004704316240162
Average distance valence: 0.07788151502609253
Average distance arousal: 0.0506197065114975
Standard deviation valence: 0.10692810267210007
Standard deviation arousal: 0.06443097442388535

******************** CLASS: Sad

Concordance: 47.31903485254691%
Cohen's Kappa: 0.3329161850500123
valence CCC: 0.6360309690227476
arousal CCC: 0.5403763103329859
Average distance valence: 0.050757430493831635
Average distance arousal: 0.06859177350997925
Standard deviation valence: 0.06478980928659439
Standard deviation arousal: 0.08127778768539429

******************** CLASS: Surprise

Concordance: 84.13705583756345%
Cohen's Kappa: 0.4051213914723999
valence CCC: 0.5782385354001861
arousal CCC: 0.7020388551214303
Average distance valence: 0.05802644416689873
Average distance arousal: 0.04756869003176689
Standard deviation valence: 0.07183613628149033
Standard deviation arousal: 0.06859678030014038

#### Top-quarter
Concordance: 65.35889433794026%
Cohen's Kappa: 0.5954930297956449
valence CCC: 0.92101527897769
arousal CCC: 0.7996307821942424
Average distance valence: 0.056574173271656036
Average distance arousal: 0.05593733489513397
Standard deviation valence: 0.07391221821308136
Standard deviation arousal: 0.0708346888422966

******************** CLASS: Anger

Concordance: 55.69948186528497%
Cohen's Kappa: 0.402892991867418
valence CCC: 0.707026467811411
arousal CCC: 0.5644754545151239
Average distance valence: 0.04622045159339905
Average distance arousal: 0.06779969483613968
Standard deviation valence: 0.05844220891594887
Standard deviation arousal: 0.07429628074169159

******************** CLASS: Disgust

Concordance: 69.10112359550563%
Cohen's Kappa: 0.47644259051286164
valence CCC: 0.7857221145420652
arousal CCC: 0.7078300608222615
Average distance valence: 0.0466245599091053
Average distance arousal: 0.04758552089333534
Standard deviation valence: 0.06338181346654892
Standard deviation arousal: 0.05592624098062515

******************** CLASS: Fear

Concordance: 52.94117647058824%
Cohen's Kappa: 0.4066327414286539
valence CCC: 0.7871496423907262
arousal CCC: 0.7115786485555479
Average distance valence: 0.06866693496704102
Average distance arousal: 0.06132233142852783
Standard deviation valence: 0.08293242752552032
Standard deviation arousal: 0.08014734089374542

******************** CLASS: Happy

Concordance: 78.79581151832461%
Cohen's Kappa: 0.48875634056474404
valence CCC: 0.8198101904790303
arousal CCC: 0.4030043017041318
Average distance valence: 0.0707918331027031
Average distance arousal: 0.0511176735162735
Standard deviation valence: 0.09459087997674942
Standard deviation arousal: 0.06471412628889084

******************** CLASS: Sad

Concordance: 48.725212464589234%
Cohen's Kappa: 0.35332935234760077
valence CCC: 0.6451503829919424
arousal CCC: 0.5686603298095708
Average distance valence: 0.051660288125276566
Average distance arousal: 0.06679018586874008
Standard deviation valence: 0.06646621972322464
Standard deviation arousal: 0.07809153944253922

******************** CLASS: Surprise

Concordance: 86.66666666666667%
Cohen's Kappa: 0.5723766734326179
valence CCC: 0.6921436466524152
arousal CCC: 0.8490777411675848
Average distance valence: 0.054210927337408066
Average distance arousal: 0.040734414011240005
Standard deviation valence: 0.06381498277187347
Standard deviation arousal: 0.05630484223365784

### Experiment1.1 (centerface)

#### all

Concordance: 59.3269330029937%
Cohen's Kappa: 0.5218865291031817
valence CCC: 0.8394614687065095
arousal CCC: 0.7293362559578545
Average distance valence: 0.06858868896961212
Average distance arousal: 0.06263934075832367
Standard deviation valence: 0.08407367020845413
Standard deviation arousal: 0.07865320891141891

******************** CLASS: Anger

Concordance: 51.37221269296741%
Cohen's Kappa: 0.3364933571255091
valence CCC: 0.5524159662310552
arousal CCC: 0.5142387011557044
Average distance valence: 0.05753841996192932
Average distance arousal: 0.07250797003507614
Standard deviation valence: 0.06905428320169449
Standard deviation arousal: 0.08424682170152664

******************** CLASS: Disgust

Concordance: 60.054844606946986%
Cohen's Kappa: 0.3813698481239026
valence CCC: 0.6198547078665357
arousal CCC: 0.5029735194762672
Average distance valence: 0.06269186735153198
Average distance arousal: 0.05661120265722275
Standard deviation valence: 0.07979614287614822
Standard deviation arousal: 0.07198700308799744

******************** CLASS: Fear

Concordance: 54.82529118136439%
Cohen's Kappa: 0.4258381034569212
valence CCC: 0.742861852458123
arousal CCC: 0.6577282233655383
Average distance valence: 0.06964762508869171
Average distance arousal: 0.06459445506334305
Standard deviation valence: 0.08838328719139099
Standard deviation arousal: 0.08292964845895767

******************** CLASS: Happy

Concordance: 70.30456852791879%
Cohen's Kappa: 0.385377300490801
valence CCC: 0.6803238454154008
arousal CCC: 0.32667194380603887
Average distance valence: 0.09736744314432144
Average distance arousal: 0.05610606446862221
Standard deviation valence: 0.12093297392129898
Standard deviation arousal: 0.06935013830661774

******************** CLASS: Neutral

Concordance: 55.32067359369401%
Cohen's Kappa: 0.3535725549208788
valence CCC: 0.5572547198438296
arousal CCC: 0.5532977644806901
Average distance valence: 0.06438068300485611
Average distance arousal: 0.06281231343746185
Standard deviation valence: 0.07424841076135635
Standard deviation arousal: 0.07612612098455429

******************** CLASS: Sad

Concordance: 50.36429872495446%
Cohen's Kappa: 0.35464215268109134
valence CCC: 0.5200430775213549
arousal CCC: 0.6151764172079466
Average distance valence: 0.06333556771278381
Average distance arousal: 0.0664152279496193
Standard deviation valence: 0.07469012588262558
Standard deviation arousal: 0.08204606175422668

******************** CLASS: Surprise

Concordance: 78.33622183708839%
Cohen's Kappa: 0.3442704923994472
valence CCC: 0.5035859442483538
arousal CCC: 0.6270155056930085
Average distance valence: 0.06993956863880157
Average distance arousal: 0.05902719125151634
Standard deviation valence: 0.07529063522815704
Standard deviation arousal: 0.0829615667462349

#### top-half
Concordance: 63.76350921805467%
Cohen's Kappa: 0.5762166357824431
valence CCC: 0.8844369332907582
arousal CCC: 0.7719355986519866
Average distance valence: 0.06859666109085083
Average distance arousal: 0.06058117747306824
Standard deviation valence: 0.08599540591239929
Standard deviation arousal: 0.0771118775010109

******************** CLASS: Anger

Concordance: 52.91925465838509%
Cohen's Kappa: 0.34481870013851157
valence CCC: 0.6011356520293133
arousal CCC: 0.5115979026565113
Average distance valence: 0.055322445929050446
Average distance arousal: 0.07226593047380447
Standard deviation valence: 0.06481830775737762
Standard deviation arousal: 0.08587073534727097

******************** CLASS: Disgust

Concordance: 64.8936170212766%
Cohen's Kappa: 0.40806998398883687
valence CCC: 0.6658458895259477
arousal CCC: 0.5548968832103881
Average distance valence: 0.05793442949652672
Average distance arousal: 0.05309091880917549
Standard deviation valence: 0.0736779049038887
Standard deviation arousal: 0.06681157648563385

******************** CLASS: Fear

Concordance: 54.55650060753341%
Cohen's Kappa: 0.42102540662449484
valence CCC: 0.7494783644724267
arousal CCC: 0.6633115497171803
Average distance valence: 0.07088664174079895
Average distance arousal: 0.06574336439371109
Standard deviation valence: 0.09107808768749237
Standard deviation arousal: 0.08417216688394547

******************** CLASS: Happy

Concordance: 74.90683229813665%
Cohen's Kappa: 0.3890080821820012
valence CCC: 0.6744417892317516
arousal CCC: 0.3554681840859956
Average distance valence: 0.09557543694972992
Average distance arousal: 0.05593780055642128
Standard deviation valence: 0.12010923027992249
Standard deviation arousal: 0.06996459513902664

******************** CLASS: Sad

Concordance: 48.525469168900806%
Cohen's Kappa: 0.3348966580451629
valence CCC: 0.5303425075680892
arousal CCC: 0.6130316112669174
Average distance valence: 0.06281422078609467
Average distance arousal: 0.06798949092626572
Standard deviation valence: 0.07326790690422058
Standard deviation arousal: 0.08243191242218018

******************** CLASS: Surprise

Concordance: 86.42131979695431%
Cohen's Kappa: 0.416950183940475
valence CCC: 0.5085123947902064
arousal CCC: 0.7114757348285954
Average distance valence: 0.06785409897565842
Average distance arousal: 0.048131052404642105
Standard deviation valence: 0.07215312123298645
Standard deviation arousal: 0.06704321503639221

#### top-quarter

Concordance: 65.4480606330807%
Cohen's Kappa: 0.596442158198992
valence CCC: 0.8952941716868464
arousal CCC: 0.8004073926856639
Average distance valence: 0.06617286056280136
Average distance arousal: 0.05799900367856026
Standard deviation valence: 0.08209137618541718
Standard deviation arousal: 0.07367497682571411

******************** CLASS: Anger

Concordance: 53.8860103626943%
Cohen's Kappa: 0.36267589303106484
valence CCC: 0.6395139544719286
arousal CCC: 0.5380529363344124
Average distance valence: 0.05403783172369003
Average distance arousal: 0.06923453509807587
Standard deviation valence: 0.061630476266145706
Standard deviation arousal: 0.08362267166376114

******************** CLASS: Disgust

Concordance: 66.57303370786516%
Cohen's Kappa: 0.44938913439043404
valence CCC: 0.7155314962937689
arousal CCC: 0.6475733319412172
Average distance valence: 0.05588136240839958
Average distance arousal: 0.04973144829273224
Standard deviation valence: 0.0721793845295906
Standard deviation arousal: 0.06261477619409561

******************** CLASS: Fear

Concordance: 55.24296675191815%
Cohen's Kappa: 0.435539753510089
valence CCC: 0.7640439666052669
arousal CCC: 0.7041433890024454
Average distance valence: 0.0681740865111351
Average distance arousal: 0.06344468891620636
Standard deviation valence: 0.08698411285877228
Standard deviation arousal: 0.08125552535057068

******************** CLASS: Happy

Concordance: 76.70157068062828%
Cohen's Kappa: 0.4431486880466473
valence CCC: 0.7390338302166102
arousal CCC: 0.365418791119748
Average distance valence: 0.0905117392539978
Average distance arousal: 0.05632098764181137
Standard deviation valence: 0.11193518340587616
Standard deviation arousal: 0.07064365595579147

******************** CLASS: Sad

Concordance: 50.991501416430594%
Cohen's Kappa: 0.3699095139340287
valence CCC: 0.5721714566799403
arousal CCC: 0.6366477985463518
Average distance valence: 0.06080588698387146
Average distance arousal: 0.06650330871343613
Standard deviation valence: 0.07177253812551498
Standard deviation arousal: 0.07960325479507446

******************** CLASS: Surprise

Concordance: 89.06666666666668%
Cohen's Kappa: 0.6152883773301638
valence CCC: 0.5959850889803148
arousal CCC: 0.8513402141923275
Average distance valence: 0.06660617887973785
Average distance arousal: 0.04230848327279091
Standard deviation valence: 0.06673786789178848
Standard deviation arousal: 0.05550388619303703